In [100]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
random.seed(1234)
import pickle
workdir = '../datasets/'
args = {'test_prop': 0.1, 'dataset': 'Gowalla'}
# from pandarallel import pandarallel
# pandarallel.initialize(nb_workers=20)

In [101]:
click_f = pd.read_table(workdir + 'Gowalla/Gowalla_totalCheckins.txt', sep='\t', names=['uid', 'check_time', 'lat', 'lng', 'locid'])
trust_f = pd.read_table(workdir + 'Gowalla/Gowalla_edges.txt', sep='\t', names=['uid', 'fid'])
click_f['rating'] = 1
#calculate rating by geo voting
#calculate rating by geo voting

In [102]:
def rating_place(item_uni):
    #如果去的地方小于5个，那就按照54321的逆序评分给定分数
    #如果去的地方大于5个，按照去的地方个数平分评分
    RATING_COUNT = 5
    TRANSFER_POSITION = 2
    # 2是为了移动索引位置: [2 3 3 4 5 5 0 0] -> [1 1 2 3 3 4 5 5]
    rating = np.zeros_like(item_uni, dtype=int)
    if len(item_uni) <= RATING_COUNT:
        last_item = item_uni[-1] + 1
        scores = 6
        for i in range(len(rating)):
            rating_ind = len(rating) - i - 1
            if item_uni[rating_ind] != last_item:
                scores = scores - 1
                rating[rating_ind] = scores
            elif item_uni[rating_ind] == last_item:
                rating[rating_ind] = rating[rating_ind + 1]
            last_item = item_uni[rating_ind]
    if len(item_uni) > RATING_COUNT:
        last_ind = 0
        for i in range(5):
            now_ind = math.floor(i * len(item_uni) / RATING_COUNT) + TRANSFER_POSITION
            rating[last_ind:now_ind] = i+1
            last_ind = now_ind
    return dict(zip(item_uni, rating))

In [97]:
example_locid = click_f.loc[click_f['uid']==7]['locid'].tolist()
(uniq, freq) = (np.unique(example_locid, return_counts=True))
# 定位id及次数
(item_uni, freq_uni) = np.unique(freq, return_counts=True)
# 次数频率统计
dict_rating = rating_place(item_uni)
print('count, rating\n去了count item 次的给rating item 分')
print(dict_rating)
dict_loctorating = dict(zip(uniq, [dict_rating[v] for v in freq]))

len(item_uni): 5
rating: [0 0 0 0 0]
count, rating
去了count item 次的给rating item 分
{1: 1, 2: 2, 3: 3, 5: 4, 21: 5}


In [46]:
_data = pd.read_csv('../datasets/Gowalla/Gowalla_rating.csv')

(array([0, 1, 2, 3, 4, 5]),
 array([ 860076, 2090828,  770323,  812504,  854447, 1054714], dtype=int64))

In [68]:
# click_f.loc[click_f['uid']==0, 'rating'] = click_f.loc[click_f['uid']==0].parallel_apply(lambda r: dict_loctorating[r['locid']], axis=1)
click_f.loc[click_f['uid']==10, 'rating'] = click_f.loc[click_f['uid']==10].apply(lambda r: dict_loctorating[r['locid']], axis=1)

In [ ]:
unique_uid = np.unique(click_f['uid'].tolist())

for uid_ind in tqdm(range(len(unique_uid))):
    uid = unique_uid[uid_ind]
    example_locid = click_f.loc[click_f['uid']==uid]['locid'].tolist()
    (uniq, freq) = (np.unique(example_locid, return_counts=True))
    # 定位id及次数
    (item_uni, freq_uni) = np.unique(freq, return_counts=True)
    # 次数频率统计
    dict_rating = rating_place(item_uni)
    dict_loctorating = dict(zip(uniq, [dict_rating[v] for v in freq]))
    # click_f.loc[click_f['uid']==uid, 'rating'] = click_f.loc[click_f['uid']==uid].parallel_apply(lambda r: dict_loctorating[r['locid']], axis=1)
    click_f.loc[click_f['uid']==uid, 'rating'] = click_f.loc[click_f['uid']==uid].apply(lambda r: dict_loctorating[r['locid']], axis=1)
    del dict_loctorating

click_f.to_csv('../datasets/Gowalla/Gowalla_rating.csv', index=False)

In [6]:
import zipfile, os
def file2zip(zip_file_name: str, file_names: list):
    with zipfile.ZipFile(zip_file_name, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
        for fn in file_names:
            parent_path, name = os.path.split(fn)
            zf.write(fn, arcname=name)

def zip2file(zip_file_name:str, extract_path: str, members=None, pwd=None):
    with zipfile.ZipFile(zip_file_name) as zf:
        zf.extractall(extract_path, members=members, pwd=pwd)

zip_name = '../datasets/Gowalla/Gowalla_rating.zip'
files = ['../datasets/Gowalla/Gowalla_rating.csv']
file2zip(zip_name, files)

In [86]:
click_f = pd.read_csv(workdir + 'Gowalla/Gowalla_rating.csv')
trust_f = pd.read_table(workdir + 'Gowalla/Gowalla_edges.txt', sep='\t', names=['uid', 'fid'])

In [87]:
click_f.loc[click_f['rating'] == 0]

,uid,check_time,lat,lng,locid,rating
2612,7,2010-10-20T18:52:30Z,30.269103,-97.749395,420315,0
2613,7,2010-10-20T15:37:34Z,30.269103,-97.749395,420315,0
2614,7,2010-10-19T18:41:01Z,30.267910,-97.749312,21714,0
2615,7,2010-10-19T15:14:33Z,30.269103,-97.749395,420315,0
2616,7,2010-10-19T01:46:04Z,30.275791,-97.752572,59824,0
...,...,...,...,...,...,...
6442402,196488,2010-10-03T09:08:30Z,51.979823,8.483964,1129217,0
6442403,196488,2010-10-03T09:07:45Z,51.979726,8.479257,4730040,0
6442404,196488,2010-10-03T09:07:31Z,51.980387,8.477926,4197024,0
6442405,196488,2010-10-01T15:29:07Z,51.979726,8.479257,4730040,0


In [32]:
np.unique(click_f['rating'].values.tolist(), return_counts=True)

(array([0, 1, 2, 3, 4, 5]),
 array([ 860076, 2090828,  770323,  812504,  854447, 1054714], dtype=int64))

In [21]:
click_f = click_f[['uid', 'locid', 'rating']]
click_f_uid = click_f['uid'].values.tolist()
click_f_locid = click_f['locid'].values.tolist()
click_f_rating = click_f['rating'].values.tolist()
click_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

user_count = 0
item_count = 0
rate_count = 0

In [22]:
for i in tqdm(range(len(click_f))):
	uid = click_f_uid[i]
	iid = click_f_locid[i]
	label = click_f_rating[i]

	if uid > user_count:
		user_count = uid
	if iid > item_count:
		item_count = iid
	if label > rate_count:
		rate_count = label
	click_list.append([uid, iid, label])

100%|██████████| 6442892/6442892 [00:06<00:00, 1057942.68it/s]


In [25]:
pos_list = []
for i in tqdm(range(len(click_list))):
	pos_list.append((click_list[i][0], click_list[i][1], click_list[i][2]))
pos_list = list(set(pos_list))

# train, valid and test data split
random.shuffle(pos_list)
num_test = int(len(pos_list) * args['test_prop'])
test_set = pos_list[:num_test]
valid_set = pos_list[num_test:2 * num_test]
train_set = pos_list[2 * num_test:]
print('Train samples: {}, Valid samples: {}, Test samples: {}'.format(len(train_set), len(valid_set), len(test_set)))
train_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
valid_df = pd.DataFrame(valid_set, columns = ['uid', 'iid', 'label'])
test_df = pd.DataFrame(test_set, columns = ['uid', 'iid', 'label'])

click_df = pd.DataFrame(click_list, columns = ['uid', 'iid', 'label'])
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'uid')

"""
u_items_list: 存储每个用户交互过的物品iid和对应的评分，没有则为[(0, 0)]
"""
for u in tqdm(range(user_count + 1)):
	hist = train_df[train_df['uid'] == u]
	u_items = hist['iid'].tolist()
	u_ratings = hist['label'].tolist()
	if u_items == []:
		u_items_list.append([(0, 0)])
	else:
		u_items_list.append([(iid, rating) for iid, rating in zip(u_items, u_ratings)])

train_df = train_df.sort_values(axis = 0, ascending = True, by = 'iid')

"""
i_users_list: 存储与每个物品相关联的用户及其评分，没有则为[(0, 0)]
"""
for i in tqdm(range(item_count + 1)):
	hist = train_df[train_df['iid'] == i]
	i_users = hist['uid'].tolist()
	i_ratings = hist['label'].tolist()
	if i_users == []:
		i_users_list.append([(0, 0)])
	else:
		i_users_list.append([(uid, rating) for uid, rating in zip(i_users, i_ratings)])

100%|██████████| 6442892/6442892 [00:02<00:00, 2776665.27it/s]


Train samples: 3185068, Valid samples: 398133, Test samples: 398133


100%|██████████| 5977758/5977758 [5:14:19<00:00, 316.96it/s]   


TypeError: '>' not supported between instances of 'str' and 'int'

In [38]:
trust_f_uid = trust_f['uid'].values.tolist()
trust_f_fid = trust_f['fid'].values.tolist()
for i in tqdm(range(len(trust_f_uid))):
	uid = trust_f_uid[i]
	fid = trust_f_fid[1]
	if uid > user_count or fid > user_count:
		continue
	trust_list.append([uid, fid])

trust_df = pd.DataFrame(trust_list, columns = ['uid', 'fid'])
trust_df = trust_df.sort_values(axis = 0, ascending = True, by = 'uid')


100%|██████████| 1900654/1900654 [00:00<00:00, 2275399.32it/s]


In [39]:
"""
u_users_list: 存储每个用户互动过的用户uid；
u_users_items_list: 存储用户每个朋友的物品iid列表
"""
for u in tqdm(range(user_count + 1)):
	hist = trust_df[trust_df['uid'] == u]
	u_users = hist['fid'].unique().tolist()
	if u_users == []:
		u_users_list.append([0])
		u_users_items_list.append([[(0,0)]])
	else:
		u_users_list.append(u_users)
		uu_items = []
		for uid in u_users:
			uu_items.append(u_items_list[uid])
		u_users_items_list.append(uu_items)

100%|██████████| 196586/196586 [06:58<00:00, 469.84it/s]
